In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from tqdm import tqdm

In [2]:
# 데이터 로드
df = pd.read_csv('preprocessed_data_ver.2.3._IRR+LOG+BINARY.csv')

In [3]:
# 모든 열 다 보기
pd.set_option('display.max_columns', None)

# 모든 행 다 보기
pd.set_option('display.max_rows', None)

# 열 너비 넉넉하게
pd.set_option('display.width', None)

# 열 안의 값 자르지 않게
pd.set_option('display.max_colwidth', None)

In [ ]:
df.head()

In [4]:
# 독립변수에서 제외할 변수 목록
drop_cols = [
    'term', 'last_pymnt_d', 'installment', 'funded_amnt',
    'recoveries', 'collection_recovery_fee', 'default', 'issue_d'
]

X = df.drop(columns=drop_cols)
y = df['default']

In [5]:
from fredapi import Fred

# FRED API 연결
fred = Fred(api_key="f9ef939a9a1eeff85d31ed2c984507d9")  # 🔑 실제 API 키로 대체해야 함

# 미국 3년 만기 국채 수익률(GS3) 시계열 불러오기
series = fred.get_series('GS3')  # pandas.Series 형식으로 반환됨


print(series.head())

1953-04-01    2.51
1953-05-01    2.72
1953-06-01    2.74
1953-07-01    2.62
1953-08-01    2.58
dtype: float64


In [ ]:
def get_nearest_rate(issue_date, rate_series):
    if pd.isnull(issue_date):
        return np.nan
    try:
        idx = rate_series.index.get_indexer([issue_date], method='nearest')[0]
        return rate_series.iloc[idx] / 100  # % 단위 → 소수로 변환
    except Exception as e:
        print(f"Error: {issue_date} ▶ {e}")
        return np.nan


df['risk_free_rate'] = df['issue_d'].apply(lambda x: get_nearest_rate(x, series))




In [8]:
df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'])
df['issue_d'] = pd.to_datetime(df['issue_d'])  # 혹시 모르니 이것도

In [9]:
print(df[['issue_d', 'risk_free_rate', 'last_pymnt_d']].head())

     issue_d  risk_free_rate last_pymnt_d
0 2017-12-01          0.0196   2020-04-01
1 2016-07-01          0.0079   2016-12-01
2 2017-10-01          0.0168   2019-08-01
3 2017-05-01          0.0148   2019-03-01
4 2015-09-01          0.0101   2018-10-01


In [11]:
from dateutil.relativedelta import relativedelta

def create_cash_flow_from_dates(row):
    try:
        term = int(row['term'])
        default = int(row['default'])
        issue_d = row['issue_d']  # 이미 datetime 형식
        last_pymnt_d = pd.to_datetime(row['last_pymnt_d'])  # 이건 문자열일 수도 있으니 변환
        installment = float(row['installment'])
        funded_amnt = float(row['funded_amnt'])
        recoveries = float(row['recoveries'])
        collection_fee = float(row['collection_recovery_fee'])

        # 첫 현금흐름: 대출 실행
        cash_flow = [-funded_amnt]

        if pd.isnull(issue_d) or pd.isnull(last_pymnt_d):
            return np.nan

        # 몇 회차까지 납입했는지 계산
        delta = relativedelta(last_pymnt_d, issue_d)
        last_pymnt_num = delta.years * 12 + delta.months

        for month in range(1, term + 1):
            if default == 1:
                if month <= last_pymnt_num:
                    cash_flow.append(installment)
                elif month == last_pymnt_num + 1:
                    cash_flow.append(recoveries - collection_fee)
                else:
                    cash_flow.append(0)
            else:
                cash_flow.append(installment)

        return cash_flow

    except Exception as e:
        print(f"[오류] index={row.name}, error={e}")
        return np.nan


In [12]:
df['cash_flow'] = df.apply(create_cash_flow_from_dates, axis=1)

In [ ]:
!pip install numpy-financial

In [13]:
import numpy_financial as npf

def get_irr(cash_flow):
    if not isinstance(cash_flow, list) or len(cash_flow) == 0:
        return np.nan
    irr_monthly = npf.irr(cash_flow)
    if irr_monthly is None or np.isnan(irr_monthly):
        return np.nan
    return (1 + irr_monthly) ** 12 - 1  # 연환산

In [14]:
df['irr'] = df['cash_flow'].apply(get_irr)

In [15]:
df['irr'] = df['irr'].fillna(df['risk_free_rate'])

df['irr'] = df['cash_flow'].apply(lambda x: np.irr(x) if isinstance(x, list) else np.nan)




---



In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [17]:
# 결과 저장용 리스트
best_thresholds = []
validation_sharpes = []
test_sharpes = []
test_approval_rates = []
test_irr_means = []
test_irr_positive_rates = []


# Sharpe 계산 함수
def calculate_sharpe(returns, risk_free_rates):
    excess = returns - risk_free_rates
    if excess.std(ddof=1) == 0:
        return -np.inf
    return excess.mean() / excess.std(ddof=1)

In [ ]:
for i in range(2):
    # Train-test-split
    # 먼저 test 데이터 분리 (20%)
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.2, random_state=i, stratify=y
    )

    # 남은 80% 중에서 train (60%) / val (20%) 분리
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.25, random_state=i, stratify=y_temp
    )
    # 랜덤 포레스트 모델 학습 (기본 하이퍼파라미터)
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # 검증 세트에 대한 부도 확률 예측
    y_pred_proba = model.predict_proba(X_val)[:, 1]

    # 사용할 threshold 후보들
    thresholds = np.arange(0.0, 0.95, 0.05)

    best_sharpe = -np.inf
    best_threshold = None

    # X_val의 인덱스를 이용해 df에서 대응하는 행 추출
    val_indices = X_val.index
    df_val = df.loc[val_indices]

    for threshold in thresholds:
        approved_mask = y_pred_proba <= threshold
        denied_mask = ~approved_mask

        selected = df_val.copy()
        
        # 승인된 건: 실제 IRR 사용
        selected.loc[approved_mask, 'irr_adj'] = selected.loc[approved_mask, 'irr']

        # 거절된 건: IRR을 국채수익률로 설정
        selected.loc[denied_mask, 'irr_adj'] = selected.loc[denied_mask, 'risk_free_rate']

        # 수익률과 무위험수익률 추출
        returns = selected['irr_adj']
        risk_free = selected['risk_free_rate']

        # NaN 제거
        valid = returns.notnull() & risk_free.notnull()
        if valid.sum() < 2:
            continue

        sharpe = calculate_sharpe(returns[valid], risk_free[valid])

        if sharpe > best_sharpe:
            best_sharpe = sharpe
            best_threshold = threshold

    # 저장
    best_thresholds.append(best_threshold)
    validation_sharpes.append(best_sharpe)

    # 4. Test 성능 측정
    y_test_proba = model.predict_proba(X_test)[:, 1]
    test_approved_mask = y_test_proba <= best_threshold
    df_test = df.loc[X_test.index]
    test_selected = df_test[test_approved_mask]

    returns_test = test_selected['irr']
    risk_free_test = test_selected['risk_free_rate']

    valid = returns_test.notnull() & risk_free_test.notnull()
    returns_test = returns_test[valid]
    risk_free_test = risk_free_test[valid]

    sharpe_test = calculate_sharpe(returns_test, risk_free_test)

    test_sharpes.append(sharpe_test)
    test_approval_rates.append(len(returns_test) / len(df_test))
    test_irr_means.append(returns_test.mean())
    test_irr_positive_rates.append((returns_test > 0).mean())

In [ ]:
print("Best thresholds from 2 runs:")
print(best_thresholds)

print("\nValidation Sharpe Ratios from 2 runs:")
print(validation_sharpes)

print("\nTest Sharpe Ratios from 2 runs:")
print(test_sharpes)

print("\nTest Approval Rates:")
print(test_approval_rates)

print("\nTest IRR Means:")
print(test_irr_means)

print("\nTest IRR Positive Rates:")
print(test_irr_positive_rates)
